## data

In [4]:
import pandas as pd
brunetti = pd.read_csv('raw-data/brunetti-length.txt',delimiter='|',header=None)

In [82]:
brunetti = brunetti.rename(columns={
    0:'chapter',
    1:'sentence',
    4:'line',
    5:'half_line',
    6:'word_number_half_line',
    8:'preceding_punct',
    9:'token_no_length',
    10:'following_punct',
    11:'word_class',
    12:'morphology',
    13:'lemma',
    14:'syntax',
    15:'homography',
    16:'modern_english',
    17:'token'
}) # still missing 2,3,7

In [83]:
brunetti

,chapter,sentence,2,3,line,half_line,word_number,7,preceding_punct,token_no_length,following_punct,word_class,morphology,lemma,syntax,homography,modern_english,token,token_w_punct,ve_ref
0,0,1,1,0,1,a,1,-,NaN,Hwæt,!,NaN,NaN,hwæt,e,NaN,well,Hwæt,Hwæt!,1.t1
1,0,1,0,0,1,a,2,-,NaN,We,NaN,NaN,np,we,p,NaN,we,Wē,Wē,1.t2
2,0,1,0,0,1,a,3,/,NaN,Gar-Dena,NaN,NaN,gp,Gar-Dene,np,NaN,Spear-Danes,Gār-Dena,Gār-Dena,1.t3
3,0,1,0,0,1,b,1,/,NaN,in,NaN,rd,NaN,in,pp,NaN,in,in,in,1.t1
4,0,1,0,0,1,b,2,-,NaN,geardagum,",",NaN,dp,gear-dagas,m,NaN,days of yore,gēardagum,"gēardagum,",1.t2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17239,43,105,0,0,3181,b,2,-,NaN,monðwærust,",",g,nsms,monn-ðwære,a,NaN,kind,monðwǣrust,"monðwǣrust,",3181.t2
17240,43,105,0,0,3182,a,1,-,NaN,leodum,NaN,NaN,dp,leod,m,NaN,man,lēodum,lēodum,3182.t1
17241,43,105,0,0,3182,a,2,/,NaN,liðost,NaN,d,nsms,liðe,a,NaN,gracious,līðost,līðost,3182.t2
17242,43,105,0,0,3182,b,1,/,NaN,ond,NaN,NaN,NaN,ond,c,NaN,and,ond,ond,3182.t1


## generate text file

In [69]:
import numpy as np
brunetti['token_w_punct'] = brunetti.apply(lambda x: x["token"]+x["following_punct"] if not isinstance(x['following_punct'],float) else x['token'],axis=1)
brunetti['token_w_punct'] = brunetti.apply(lambda x: x["preceding_punct"]+x["token_w_punct"] if not isinstance(x['preceding_punct'],float) else x['token_w_punct'],axis=1)

In [70]:
brunetti['token_w_punct']

0               Hwæt!
1                  Wē
2            Gār-Dena
3                  in
4          gēardagum,
             ...     
17239     monðwǣrust,
17240          lēodum
17241          līðost
17242             ond
17243    lofgeornost.
Name: token_w_punct, Length: 17244, dtype: object

In [71]:
lines = []
for i in brunetti['line'].unique():
    token_list = brunetti.loc[brunetti.line == i].token_w_punct.to_list()
    full_line = ' '.join(token_list)
    lines.append(full_line)

In [73]:
with open('beowulf_with_line_numbers.txt','w') as b:
    for i,l in enumerate(lines):
        b.write(f'{i+1} {l}\n')

## generate csv

In [86]:
word_count_per_line = []
for i in brunetti['line'].unique():
    for i,token in enumerate(brunetti.loc[brunetti.line == i].token_w_punct.to_list()):
        word_count_per_line.append(i+1)
        
brunetti['word_number_line'] = word_count_per_line

In [88]:
brunetti['ve_ref'] = brunetti.apply(lambda x: f"{x['line']}.t{x['word_number_line']}",axis=1)
brunetti_csv = brunetti[['ve_ref','token_w_punct','token','lemma','syntax','modern_english','morphology']]

In [95]:
brunetti_csv.loc[brunetti.morphology.isnull(), 'morphology'] = 'indeclinable'

/var/folders/l_/b5wczsf55z9dmf_dzr3vw6ph0000gr/T/ipykernel_67398/1983220175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brunetti_csv.loc[brunetti.morphology.isnull(), 'morphology'] = 'indeclinable'


In [98]:
brunetti_csv.to_csv('beowulf_annotations.csv',index=False)